In [1]:
import os, shutil, itertools, yaml, kornia, torchvision, sys, copy, math
from functools import partial
import dill as pickle
from importlib import reload
osp = os.path
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
import torch
nn = torch.nn
F = nn.functional
import monai.transforms as mtr

In [2]:
from inrnet import args as args_module
from inrnet import jobs as job_mgmt
from inrnet import optim, util, losses
from inrnet.data import dataloader, kitti
from inrnet.experiments import depth
from inrnet import inn
import inrnet.inn.functional as inrF
%matplotlib inline
plt.rcParams["figure.figsize"] = (4.0, 3.0)
plt.rcParams['figure.dpi'] = 200

rescale_clip = mtr.ScaleIntensityRangePercentiles(lower=1, upper=99, b_min=0, b_max=255, clip=True, dtype=np.uint8)
rescale_noclip = mtr.ScaleIntensityRangePercentiles(lower=0, upper=100, b_min=0, b_max=255, clip=False, dtype=np.uint8)
rescale_float = mtr.ScaleIntensity()

TMP_DIR=osp.expanduser("~/code/diffcoord/temp")

rsync -av --ignore-existing \
clintonw@peppercorn.csail.mit.edu:/data/vision/polina/users/clintonw/code/placenta/results/unetr_00 Downloads/

## interactive

In [ ]:
model = torchvision.models.vgg11_bn(pretrained=True)

In [15]:
model.features[0]

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): ReLU(inplace=True)
  (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): ReLU(inplace=True)
  (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (13): ReLU(inplace=True)
  (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, cei

In [4]:
from inrnet.models.inrs.siren import to_black_box
args = job_mgmt.get_job_args("dep1")
data_loader = dataloader.get_inr_dataloader(args["data loading"])
for inr, _ in data_loader:
    inr = to_black_box(inr)
    break

In [5]:
img_shape = h,w = 352//32, 1216//32
with torch.cuda.amp.autocast():
    img = inr.cuda().produce_image(h,w)
x = torch.tensor(img).permute(2,0,1).unsqueeze(0).cuda()

In [6]:
relu = model.eval().features[1]
InrNet = inn.conversion.translate_discrete_layer(relu)
y = relu(x)

In [7]:
coords = util.meshgrid_coords(*x.shape[-2:])
out_inr = InrNet(inr)
with torch.no_grad():
    with torch.cuda.amp.autocast():
        out = out_inr.eval()(coords)
        out = util.realign_values(out, coords_gt=coords, inr=out_inr)
        out = out.reshape(h,w,3).permute(2,0,1).unsqueeze(0)
        #img = out_inr.cuda().produce_image(h,w)

In [10]:
out_inr = InrNet(inr)

In [ ]:
out_inr.sampled_coords

In [ ]:
print(y.shape == out.shape)
torch.allclose(y.half(), out.half())

In [374]:
base_dir = "/data/vision/polina/scratch/clintonw/datasets/imagenet21k_resized/imagenet21k_train"
subpaths = util.glob2(base_dir+"/n06275353")

In [ ]:
plt.imshow(plt.imread(subpaths[22]))

In [ ]:
subpaths[:10]

In [ ]:
plt.imshow(rescale_float(img2), cmap="gray")

In [ ]:
plt.imshow(rescale_float(img.squeeze().numpy()), cmap="gray")

In [ ]:
python train.py -j=a6 -c=a6

In [26]:
h,w=8,8
xy_grid = util.meshgrid_coords(h,w)

In [97]:
import torchvision.models as models
m = models.efficientnet_b0()

In [4]:
from inrnet.models.inrs import dvr

In [15]:
D = dvr.Decoder(out_dim=1, c_dim=256)
M = dvr.DVR(D)

In [5]:
sd = torch.load(osp.expanduser("~/code/differentiable_volumetric_rendering/models/ours_depth-64d35726.pt"))

In [29]:
paths = util.glob2("/data/vision/polina/scratch/clintonw/datasets/ShapeNetCore.v2/02691156")

In [56]:
REND_DIR = "/data/vision/polina/scratch/clintonw/datasets/ShapeNetRendering"
imgs = util.glob2(REND_DIR+"/02691156/c2f43dd2c483e0d86b7bd17e458d0dcb/rendering/*.png")

In [40]:
imgs = util.glob2("~/code/differentiable_volumetric_rendering/media/demo/choy_renderings")

In [ ]:
M.load_state_dict(sd["model"], strict=False);

In [27]:
with torch.no_grad():
    out = M.decoder(torch.zeros(1,3))

In [197]:
layer = nn.Conv2d(4,5,3)

In [ ]:
inputs = feature_extractor(images=image, return_tensors="pt")
outputs = model(**inputs)
logits = outputs.logits

In [76]:
coco_len = 118287
kitti_len = 80896
horse_len = 1067
zebra_len = 1334

In [ ]:
for ix in range(0,coco_len,1600):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/coco/siren_{ix+63}.pt"):
        print("sh fit_inr.sh", f"coco{ix//1600}", "fit_coco", ix)

In [ ]:
for ix in range(0,kitti_len,800):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/kitti/siren_{ix+63}.pt"):
        print("sh fit_inr.sh", f"skit{ix//800}", "fit_kitti", ix)

In [ ]:
for ix in range(0,zebra_len,128):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/zebra/siren_{ix+15}.pt"):
        print("sh fit_inr.sh", f"zebra{ix//128}", "fit_zebra", ix)

In [ ]:
for ix in range(0,horse_len,128):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/horse/siren_{ix+15}.pt"):
        print("sh fit_inr.sh", f"horse{ix//128}", "fit_horse", ix)

In [6]:
coords_out.shape

torch.Size([7, 2])

In [ ]:
img = plt.imread("/data/vision/polina/scratch/clintonw/datasets/inrnet/horse2zebra/testA/n02381460_9260.jpg")

In [ ]:
coords_out = coords_out[:,0]*5 + coords_out[:,1]

In [7]:
coords_gt = coords_gt[:,0]*5 + coords_gt[:,1]
coords_out = coords_out[:,0]*5 + coords_out[:,1]

In [14]:
matches = coords_gt.unsqueeze(1) == coords_out.unsqueeze(0)

In [11]:
out

tensor([[-1.7325,  1.4917,  1.0375,  0.0800],
        [ 0.6882, -2.1554, -1.8872,  0.3857],
        [ 1.3248,  2.2584, -0.5773,  1.0302],
        [-1.3867, -2.4084, -0.1861,  0.1831],
        [ 0.6005,  0.0305, -1.4584,  0.5626],
        [ 0.1056,  0.1705,  0.1341, -1.4289],
        [-1.2510,  0.6950,  1.3818,  0.4026]])

In [13]:
coords_gt

tensor([ 4.0818,  5.8486, -7.3113,  2.7945, -4.1563,  2.0609, -3.9877])

In [12]:
coords_out

tensor([ 4.0818, -7.3113, -4.1563, -3.9877,  5.8486,  2.7945,  2.0609])

In [15]:
out[torch.where(matches)[1]]

tensor([[-1.7325,  1.4917,  1.0375,  0.0800],
        [ 0.6005,  0.0305, -1.4584,  0.5626],
        [ 0.6882, -2.1554, -1.8872,  0.3857],
        [ 0.1056,  0.1705,  0.1341, -1.4289],
        [ 1.3248,  2.2584, -0.5773,  1.0302],
        [-1.2510,  0.6950,  1.3818,  0.4026],
        [-1.3867, -2.4084, -0.1861,  0.1831]])

In [ ]:
plt.imshow(img)

In [ ]:
"/data/vision/polina/scratch/clintonw/datasets/coco/annotations/panoptic_train2017"

In [ ]:
osp.exists(f"/data/vision/polina/users/clintonw/code/diffcoord/temp/kitti/siren_{ix+63}.pt")

In [60]:
paths = sorted(util.glob2("/data/vision/polina/scratch/clintonw/datasets/inrnet/kitti/siren_*.pt"))

In [ ]:
## search python strings
import os
osp = os.path
root = osp.expanduser("~/code/placenta/placenta")
query_string = 'load_img'
for folder, subfolders, files in os.walk(root):
    for file in files:
        if file.endswith(".py"):
            path = osp.join(folder, file)
            with open(path, 'r') as f:
                if query_string in f.read():
                    print(path)

In [ ]:
coords_gt.shape

In [22]:
out

[4.0818400382995605,
 -7.311278820037842,
 -4.156335830688477,
 -3.98771595954895,
 5.848553657531738,
 2.794525623321533,
 2.060873031616211]